In [1]:
import sys
sys.path.insert(0, 'C:/Users/MyPC/Desktop/git/AVATAR_motionMap')
import FileManager.csvload as acl
sys.path.insert(0, 'C:/Users/MyPC/Desktop/git/kp_moseq/keypoint-moseq/project')
import SDSBD_trainset2.params as params

import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import time
from datetime import date
import os
import keypoint_moseq as kpms
import keypoint_moseq .analysis as kpmsa

filelist_postSD_defeated = acl.get_csv_paths(params.postSD_defeated)    #[]
filelist_postSD_control = acl.get_csv_paths(params.postSD_control)
start = params.start
end = params.end


# read "sylable reindexed" column from each csv file and store it in a dictionary
columns = ['file_name', 'syllables_reindexed', 'syllables_frequency']
data_moseq = pd.DataFrame(columns=columns)

for file in filelist_postSD_defeated:
    file_name = os.path.splitext(os.path.basename(file))[0]
    # Read the csv file
    df = pd.read_csv(file)
    df = df[start:end]

    syllables_reindexed = df['syllables reindexed']
    syllables_frequency = syllables_reindexed.value_counts(normalize=True)
    
    # Create a dictionary with the data for the current file
    row_data = {
        'file_name': file_name,
        'syllables_reindexed': syllables_reindexed, 
        'syllables_frequency': syllables_frequency
    }
    
    # Append the data for the current file to the DataFrame
    data_moseq = data_moseq.append(row_data, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [21]:
import sys
sys.path.insert(0, 'C:/Users/MyPC/Desktop/git/AVATAR_motionMap')
import FileManager.csvload as acl
sys.path.insert(0, 'C:/Users/MyPC/Desktop/git/kp_moseq/keypoint-moseq/project')
import SDSBD_trainset2.params as params

import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import time
from datetime import date
import os
import keypoint_moseq as kpms
import keypoint_moseq .analysis as kpmsa

# parameters setting
start = params.start    # analysis start frame of the video
end = params.end        # analysis end frame of the video

filelist_group1 = acl.get_csv_paths(params.postSD_defeated)    #[]
filelist_group2 = acl.get_csv_paths(params.postSD_control)
dataset = filelist_group1 + filelist_group2
dataset_class = [0]*len(filelist_group1) + [1]*len(filelist_group2)
if len(dataset) != len(dataset_class):
    print('Error: dataset and dataset_class should have the same length')
    sys.exit()


# read "sylable reindexed" column from each csv file and store it in a dictionary
data_moseq = {}  # Initialize an empty dictionary
for file, group in zip(dataset, dataset_class):
    file_name = os.path.splitext(os.path.basename(file))[0]
    data_moseq[file_name] = {}  # Create a new dictionary for each file with the file as the key
    data_moseq[file_name]['class'] = group  # Store the class of the file in the dictionary
    
    # Read the csv file
    df = pd.read_csv(file)
    df = df[start:end]
    
    # Convert the 'syllables reindexed' column to a numpy array, wrap it in a list, and store it in the dictionary
    data_moseq[file_name]['syllables reindexed'] = [df['syllables reindexed'].to_numpy(dtype=int)]
    frequencies, durations = kpmsa.get_syllable_statistics(data_moseq[file_name]['syllables reindexed'], max_syllable=100, count='frequency')

    # Calculate the relative frequencies
    total_frequency = sum(frequencies.values())
    relative_frequencies = {k: v / total_frequency for k, v in frequencies.items()}

    # Store relative frequencies in the dictionary
    data_moseq[file_name]['syllables relative frequency'] = relative_frequencies
    data_moseq[file_name]['syllables durations'] = durations

QDA (Quadratic Discriminant Analysis) model training

In [24]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
# parameter setting
test_size = 0.2 # train vs. test

# Prepare data
X = []  # Features
y = []  # Response
for file_name, data in data_moseq.items():
    relative_frequency_values = list(data['syllables relative frequency'].values())
    X.append(relative_frequency_values)
    response_value = data['class']
    y.append(response_value)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Repeated Stratified K-Fold 객체 생성
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# 분석기 초기화
qda = QuadraticDiscriminantAnalysis()

# 결과 저장을 위한 리스트
scores = []

# 교차 검증 수행
for train_index, test_index in rskf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    qda.fit(X_train, y_train) # 모델 훈련
    predictions = qda.predict(X_test) # 예측
    score = accuracy_score(y_test, predictions) # 정확도 계산
    scores.append(score)

# 평균 정확도 출력
print("Average accuracy: ", np.mean(scores))

Model accuracy: 0.7333333333333333


Variables are collinear